<center><h2>LinearPy</center>

**Table of Contents**

[1. Introduction](#1.-Introduction)
  - [1.1. Introduction](#1.1.-Introduction)
  - [1.2. Utility Functions](#1.2.-Utility-Functions)

[2. Linear Regression](#2.-Linear-Regression)
  - [2.1. Linear Modeling](#2.1.-Linear-Modeling)
  - [2.2. Closed Form Solution: Normal Equation](#2.2.-Closed-Form-Solution:-Normal-Equation)
  - [2.3. Batch Gradient Descent (with Regularization)](#2.3.-Batch-Gradient-Descent-(with-Regularization))
  - [2.4. Mini Batch Gradient Descent (with Regularization & Learning Schedule)](#2.4.-Mini-Batch-Gradient-Descent-(with-Regularization-&-Learning-Schedule))
  - [2.5. Stochastic Gradient Descent (with Regularization & Learning Schedule)](#2.5.-Stochastic-Gradient-Descent-(with-Regularization-&-Learning-Schedule))

[3. Linear Classification](#3.-Linear-Classification)
  - [3.1. Logistic Regression (with Regularization & Learning Schedule)](#3.1.-Logistic-Regression-(with-Regularization-&-Learning-Schedule))
  - [3.2. Softmax Regression (with Regularization & Learning Schedule)](#3.2.-Softmax-Regression-(with-Regularization-&-Learning-Schedule))

## 1. Introduction

### 1.1. Introduction

In [12]:
import numpy as np

This notebook presents a structured walkthrough of **Linear Modeling Techniques**, moving from theoretical foundations and mathematical formulations to practical implementations—each model is built as a custom predictor class designed to align with the style of **Scikit-Learn** (though the library itself is not used).


By implementing and inspecting these models from scratch, we gain a deeper intuition for how supervised learning tasks work under the hood. Linear models form one of the most important foundations in machine learning, powering both traditional algorithms and modern deep learning architectures.

We begin with simple **Linear Regression**, then introducing **Gradient Descent**, an **Iterative Optimization technique**  and its variants (batch, mini-batch, and stochastic). From there, we move to **Linear Classification**, covering both **Logistic Regression** (for binary classification) and **Softmax Regression** (for multi-class problems).

To enhance learning and model performance, we also explore techniques like **Regularization** and **Learning Rate Schedules**, which are critical for controlling overfitting and improving optimization.

---
**Project Requirements**

The only dependency for this project is **NumPy**, a powerful library for numerical computing and linear algebra. All implementations—from data preprocessing to model evaluation—are built entirely using NumPy, with no external ML libraries.

---
**Project Structure**

Since Scikit-Learn is not used, I have implemented several utility functions to support model evaluation and experimentation:

* **Synthetic Data Generation**

  * Functions to generate polynomial, non-linear, and multi-class datasets for both regression and classification tasks.
   

* **Preprocessing Utilities**

  * `StandardScaler` for feature normalization.
  * `OneHotEncoder` for categorical targets in classification.
  * `train_test_split` for test/val partitioning.

* **Evaluation Metrics**

  * Regression: `MSE`, `RMSE`, `MAE`
  * Classification: `Accuracy`, `Precision`, `Recall`, `F1 Score`
  * A general `ModelEvaluator` class to summarize results

### 1.2. Utility Functions

**Dataset Generation**

To generate data for evaluating models, I used a simple polynomial formula with added noise to simulate real-world conditions:

- **Regression**  
  $x_{\text{poly}} = x^2$  
  $y = w \cdot x_{\text{poly}} + \varepsilon$

- **Classification**  
  $x_{\text{poly}} = x^2$  
  $\text{logits} = W \cdot x_{\text{poly}} + \varepsilon$  
      <i>If binary:</i> $y = \text{sigmoid}(\text{logits})$  
      <i>If multiclass:</i> $y = \text{softmax}(\text{logits})$

*Where:*

$w, W$ = randomly initialized weights  
$\varepsilon$ = random noise (Gaussian)  

In [13]:
# function to generate a polynomial degree-2 regression dataset

def generate_poly_regression_data(n_samples=700, n_features=10, noise_std=1.0, random_state=42):
    """ generates a polynomial degree-2 regression dataset """
    np.random.seed(random_state)
    X = 6 * np.random.rand(n_samples, n_features)
    
    X_poly = np.c_[X, X**2] # degree-2 poly
    
    coef = np.random.randn(X_poly.shape[1])
    
    y = X_poly @ coef + np.random.normal(0, noise_std, size=n_samples) # final equation
    
    return X_poly, y

In [14]:
# function to generate a polynomial degree-2 classification dataset

def generate_poly_classification_data(n_samples=700, n_features=10, n_classes=3, noise_std=1.0, random_state=42):
    """ generates a polynomial degree-2 classification dataset """
    np.random.seed(random_state)
    
    X = 6 * np.random.rand(n_samples, n_features)
     
    X_poly = np.c_[X, X**2] # degree-2 poly
    
    coef = np.random.randn(X_poly.shape[1], n_classes)
    
    logits = X_poly @ coef + np.random.normal(0, noise_std, size=(n_samples, n_classes))
    
    if n_classes == 2:
        probs = 1 / (1 + np.exp(-logits[:, 0]))  # Sigmoid
        y = (probs >= 0.5).astype(int)
    else:
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
        probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)  # Softmax
        y = np.argmax(probs, axis=1)
    return X_poly, y

In [15]:
class StandardScaler:
    """ normalizes matrix X by standard scaling"""
    def __init__(self):
        self.mean = None

    def fit(self,X):
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)
        return self

    def transform(self,X):
        return (X - self.mean) / self.std  

In [16]:
def add_bias(X):
    """ adds a bias/y-intercept to matrix X"""
    return np.c_[np.ones(X.shape[0]) , X] #

In [17]:
def OneHotEncoding(X):
    ohe = np.zeros((X.size,X.max() + 1))
    ohe[np.arange(X.size) , X] = 1
    return ohe

In [18]:
# Regression Performance Metrics

def mean_squared_error(y_true,y_pred):
    return np.mean((y_pred.reshape(-1,1) - y_true.reshape(-1,1))**2)

def root_mean_squared_error(y_true,y_pred):
    return np.sqrt(np.mean((y_pred.reshape(-1,1) - y_true.reshape(-1,1))**2))

def mean_absolute_error(y_true,y_pred):
    return np.mean(np.absolute(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))

def r2_score(y_true,y_pred):
    return 1 - ( np.sum((y_pred.reshape(-1,1) - y_true.reshape(-1,1))**2) / np.sum((y_true.reshape(-1,1) - np.mean(y_true.reshape(-1,1)))**2) )

In [19]:
# Classidication Performance Metrics (for both binary and mulit-class classification)

def accuracy_score(y_true, y_pred):
    y_true, y_pred = np.ravel(y_true), np.ravel(y_pred)
    return np.mean(y_true == y_pred)

def precision(y_true, y_pred, average='binary'):
    y_true, y_pred = np.ravel(y_true), np.ravel(y_pred)
    labels = np.unique(np.concatenate((y_true, y_pred)))
    precisions = []
    for label in labels:
        tp = np.sum((y_pred == label) & (y_true == label))
        fp = np.sum((y_pred == label) & (y_true != label))
        precisions.append(tp / (tp + fp) if (tp + fp) > 0 else 0.0)
    if average == 'binary':
        return precisions[1] if len(precisions) > 1 else precisions[0]
    return np.mean(precisions)

def recall(y_true, y_pred, average='binary'):
    y_true, y_pred = np.ravel(y_true), np.ravel(y_pred)
    labels = np.unique(np.concatenate((y_true, y_pred)))
    recalls = []
    for label in labels:
        tp = np.sum((y_pred == label) & (y_true == label))
        fn = np.sum((y_pred != label) & (y_true == label))
        recalls.append(tp / (tp + fn) if (tp + fn) > 0 else 0.0)
    if average == 'binary':
        return recalls[1] if len(recalls) > 1 else recalls[0]
    return np.mean(recalls)

def f1_score(y_true, y_pred, average='binary'):
    p = precision(y_true, y_pred, average)
    r = recall(y_true, y_pred, average)
    return 2 * p * r / (p + r) if (p + r) > 0 else 0.0

In [20]:
# randomly sampling a val/test set

def train_test_split(X,y,random_state=42, train_size=0.8):
    np.random.seed(random_state)
    rnd_indx = np.random.permutation(len(X)) # Shuffling indexes
    
    train_end_size = int(len(X) * train_size)
    
    X_train,y_train = X[rnd_indx[:train_end_size]] , y[rnd_indx[:train_end_size]] 
    X_test,y_test = X[rnd_indx[train_end_size:]] , y[rnd_indx[train_end_size:]] 
    
    return X_train,y_train,X_test,y_test

In [29]:
def evaluate_regression(model, X_train, y_train, X_val, y_val):
    """ returns train/val regression metrics """
    print("-" * 35)
    print(f"Val Target True Mean: {y_val.mean():.4f} | Std Dev: {y_val.std():.4f}")
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    print(f"RMSE: {root_mean_squared_error(y_val, y_pred):.4f}")
    print(f"MAE: {mean_absolute_error(y_val, y_pred):.4f}")
    print(f"R² Score: {r2_score(y_val, y_pred):.4f}")
    print("-" * 35)

In [22]:
def evaluate_classification(model, X_train, y_train, X_val, y_val, average="macro"):
    """ returns train/val classificaion metrics. supports macro-average scores for multi-class """
    print("-" * 35)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    print(f"Accuracy:  {accuracy_score(y_val, y_pred):.4f}")
    print(f"Precision: {precision(y_val, y_pred, average):.4f}")
    print(f"Recall:    {recall(y_val, y_pred, average):.4f}")
    print(f"F1 Score:  {f1_score(y_val, y_pred, average):.4f}")
    
    print("-" * 35)

## 2. Linear Regression

### 2.1. Linear Modeling

**Introduction to Linear Modeling**

Linear modeling is one of the most basic and widely used methods in supervised machine learning. It aims to capture the relationship between input features and a target variable by assuming this relationship can be approximated as a weighted sum of the features.

Given input features $\mathbf{X} = [x_1, x_2, \dots, x_n]$ and parameters (weights) $\boldsymbol{\theta} = [\theta_0, \theta_1, \dots, \theta_n]^T$, the model predicts an output $\hat{y}$ as:  

$$
\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n
$$  

Here, $\theta_0$ is the bias term (or intercept), which allows the model to shift predictions up or down regardless of the input features. The other parameters $\theta_1, \dots, \theta_n$ represent how much each feature contributes to the prediction. 

The "linear" aspect refers to the fact that the prediction is a linear combination of these parameters. Importantly, the features themselves don’t have to be simple — they can be transformed or expanded into nonlinear functions like squares or sine waves, as long as the model remains linear in terms of $\boldsymbol{\theta}$. This flexibility lets linear models handle a surprising range of problems while keeping the math and computation manageable.

Example: Imagine you want to estimate house prices based on features like the size of the house ($x_1$) and the number of bedrooms ($x_2$). A linear model expresses the predicted price $\hat{y}$ as:  

$$
\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2
$$  

The weights $\theta_1$ and $\theta_2$ quantify how much each feature influences the price, while $\theta_0$ adjusts the baseline price independent of those features.

---

Formulation of Linear Regression

Linear regression is a key example of linear modeling where the goal is to find the best-fitting parameters $\boldsymbol{\theta}$ that minimize the difference between the model’s predictions and the actual target values. This difference is often measured by the mean squared error (MSE), which penalizes larger errors more heavily.

For a dataset with $m$ samples and $n$ features, we organize the input into a design matrix $\mathbf{X}$, which includes a column of ones to account for the bias term. The prediction for all samples at once can be written as:

$$
\mathbf{\hat{y}} = \mathbf{X} \boldsymbol{\theta}
$$

The task of training the model is then to find the $\boldsymbol{\theta}$ that minimizes the overall error between these predictions $\mathbf{\hat{y}}$ and the true target values $\mathbf{y}$.

This approach forms the foundation for many machine learning models and is often the first step before moving on to more complex methods.


In [25]:
# Data prepartion

# Generating data
X , y = generate_poly_regression_data(n_samples=3000, n_features=10, noise_std=1.0, random_state=42)

# train/val split
X_train , y_train  , X_val , y_val = train_test_split( X , y , random_state=42, train_size=0.8)

# scaling will help faster convergence
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

### 2.2. Closed Form Solution: Normal Equation

**Solving Linear Regression via Singular Value Decomposition (SVD)**

Given a dataset with $m$ samples and $n$ features, linear regression aims to find a parameter vector $\boldsymbol{\theta} \in \mathbb{R}^{n+1}$ (including the bias term $\theta_0$) that best fits the data. This means minimizing the mean squared error (MSE) between the predicted values $\mathbf{\hat{y}} = \mathbf{X}\boldsymbol{\theta}$ and the true targets $\mathbf{y} \in \mathbb{R}^m$. Here, the design matrix $\mathbf{X} \in \mathbb{R}^{m \times (n+1)}$ is the original features with an added column of ones to include the intercept. (What `add_bias` function does)

The problem can be expressed as:

$$
\min_{\boldsymbol{\theta}} \sum_{i=1}^m \left( (\mathbf{X}\boldsymbol{\theta})_i - y_i \right)^2
$$


When the matrix $\mathbf{X}$ is full rank (meaning $\mathbf{X}^T \mathbf{X}$ is invertible), we can find the exact solution using the normal equation:

$$
\boldsymbol{\theta} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$

However, this approach breaks down if $\mathbf{X}$ is rank-deficient (for example, if features are highly correlated or if there are fewer samples than features), because then $\mathbf{X}^T \mathbf{X}$ becomes singular and cannot be inverted.

To handle such cases more reliably, we use the Moore-Penrose pseudoinverse computed through Singular Value Decomposition (SVD). The SVD factorizes $\mathbf{X}$ into three matrices:

$$
\mathbf{X} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^T
$$

where:  
- $\mathbf{U} \in \mathbb{R}^{m \times m}$ and $\mathbf{V} \in \mathbb{R}^{(n+1) \times (n+1)}$ are orthogonal matrices,  
- $\mathbf{\Sigma} \in \mathbb{R}^{m \times (n+1)}$ is a diagonal matrix containing singular values $\sigma_i \geq 0$.  

The pseudoinverse $\mathbf{X}^+$ is then computed as:

$$
\mathbf{X}^+ = \mathbf{V} \mathbf{\Sigma}^+ \mathbf{U}^T
$$

where $\mathbf{\Sigma}^+$ is formed by taking the reciprocal of each non-zero singular value $\sigma_i$ and transposing the matrix. Using this pseudoinverse, the optimal parameters are:

$$
\boldsymbol{\theta} = \mathbf{X}^+ \mathbf{y}
$$

---

Numpy’s linear algebra module provides an implementation of the pseudoinverse via `np.linalg.pinv()`. Below is a straightforward implementation using SVD to solve linear regression with this approach.


In [26]:
class LinearRegressionSVD:
    """
    Linear Regression model solved using Singular Value Decomposition (SVD).
    This implementation uses the Moore-Penrose pseudoinverse to find the optimal parameters.
    """

    def __init__(self): 
        self.theta = None
    
    def fit(self, X, y):
        y = y.reshape(-1, 1)
        X = add_bias(X)  # Add the bias term to the design matrix
        self.theta = np.linalg.pinv(X) @ y  # Calculate theta using the Moore-Penrose pseudoinverse of X
        return self

    def predict(self, X):
        X = add_bias(X)
        yhat = X @ self.theta  # Perform prediction using the learned parameters
        return yhat

In [30]:
# Testing & Evaluating the model class

linreg_svd = LinearRegressionSVD()

evaluate_regression(linreg_svd, X_train, y_train, X_val, y_val)

-----------------------------------
Val Target True Mean: 57.8266 | Std Dev: 31.1542
RMSE: 0.9922
MAE: 0.7956
R² Score: 0.9990
-----------------------------------


### 2.3. Batch Gradient Descent (with Regularization)

**Gradient Descent: An Iterative Approach to Linear Regression**

Another way to find the parameters of a linear regression model is **gradient descent (GD)**, an iterative optimization method. Instead of directly solving for \$\theta\$, GD minimizes the loss function step-by-step.

The objective is to minimize the mean squared error (MSE):

$$
J(\theta) = \frac{1}{m} \sum_{i=1}^m (\hat{y}_i - y_i)^2 = \frac{1}{m} \|X \theta - y\|^2
$$

To minimize \$J(\theta)\$, we compute its derivative (gradient) with respect to \$\theta\$:

$$
\nabla_\theta J = \frac{2}{m} X^T (X \theta - y)
$$

Gradient descent updates parameters by moving opposite to the gradient:

$$
\theta := \theta - \eta \nabla_\theta J
$$

where \$\eta\$ is the learning rate controlling the update size.

---

**Regularization: Controlling Model Complexity**

To prevent overfitting and improve generalization, we add a penalty term to the loss, called **regularization**. This discourages large or complex parameter values.

Two common types of regularization are:

* **L1 regularization (Lasso)**: adds the sum of absolute values \$|\theta|\$, encouraging sparsity.
* **L2 regularization (Ridge)**: adds the sum of squared values \$\theta^2\$, encouraging small weights.

We can combine both in **Elastic Net** regularization:

$$
J(\theta) = \frac{1}{m} \|X \theta - y\|^2 + \lambda \left( \alpha |\theta| + (1-\alpha) \theta^2 \right)
$$

where \$\lambda\$ controls how strong the penalty is, and \$\alpha\$ balances between L1 and L2.

Taking the derivative of this full loss gives the gradient:

$$
\nabla_\theta J = \frac{2}{m} X^T (X \theta - y) + \lambda \left( \alpha \cdot \text{sign}(\theta) + 2(1-\alpha) \theta \right)
$$

Gradient descent uses this gradient to iteratively update \$\theta\$, shrinking weights to avoid overfitting. This is the key formula behind the update step in the `BatchGradientDescent` class.

In [32]:
class BatchGradientDescent:
    """
    Batch Gradient Descent with Elastic Net Regularization for linear regression.
    
    Parameters:
    -----------
    eta : float, default=0.01
        Learning rate (step size) for gradient descent.
        Must be > 0. Too large may cause divergence, too small slows convergence.
        
    n_epochs : int, default=1000
        Number of training iterations over the entire dataset.
        Must be >= 1. Higher values may lead to better convergence but longer training.
        
    alpha : float, default=0
        Mixing parameter for Elastic Net regularization:
        - alpha=0: Pure L2 (Ridge) regularization
        - alpha=1: Pure L1 (Lasso) regularization
        - 0 < alpha < 1: Elastic Net mix
        Must be between 0 and 1 (inclusive).
        
    lambda_ : float, default=0.01
        Regularization strength.
        Must be >= 0. Higher values increase regularization effect.
        
    random_state : int, default=42
        Seed for random weight initialization.
        Provides reproducibility of results.
        
    Attributes:
    -----------
    theta : ndarray of shape (n_features + 1, 1)
        Learned weight vector (includes bias term).
    """
    def __init__(self,eta=0.01,n_epochs=1000,alpha=0,lambda_=0.01, random_state=42):
        self.eta = eta
        self.n_epochs = n_epochs
        self.alpha = alpha
        self.lambda_ = lambda_
        self.random_state = random_state
        
    def fit(self,X,y):
        X = add_bias(X)
        y = y.reshape(-1,1)
        
        np.random.seed(self.random_state)
        self.theta = np.random.rand(X.shape[1],1)
        
        for epoch in range(self.n_epochs):

            l1_grad = self.lambda_ * self.alpha * np.sign(self.theta)
            l2_grad = self.lambda_ * (1-self.alpha) * 2 * self.theta
            
            gradients = (2 / X.shape[0]) * X.T @ (X @ self.theta - y) + l1_grad + l2_grad # gradient vector for MSE + ElasticNet Regularizer
            self.theta = self.theta - self.eta * gradients
        return self

    def predict(self,X):
        X = add_bias(X)
        yhat = X @ self.theta
        return yhat.reshape(-1,1)

In [33]:
# Testing & Evaluating the model class

batchGD = BatchGradientDescent(eta=0.01,n_epochs=4000,alpha=0,lambda_=0.01, random_state=42)

evaluate_regression(batchGD, X_train, y_train, X_val, y_val)

-----------------------------------
Val Target True Mean: 57.8266 | Std Dev: 31.1542
RMSE: 1.6950
MAE: 1.3607
R² Score: 0.9970
-----------------------------------


### 2.4. Mini Batch Gradient Descent (with Regularization & Learning Schedule)

Mini-batch Gradient Descent (GD) is a variant of Gradient Descent where, in each iteration, parameters are updated based on a small subset (batch) of the training dataset, rather than the entire set.

This approach is used for several reasons, including **out-of-core learning**: when the full dataset is too large to fit into memory, mini-batch GD allows processing only a portion of the data at a time, calculating parameter updates based solely on those samples.

Unlike standard GD, where the cost function typically decreases smoothly toward the minimum, mini-batch GD causes the cost function to fluctuate, decreasing only on average. Over time, it approaches the minimum but continues to oscillate around it, never fully settling. As a result, the final parameter values are generally good but not optimal.

**Learning Schedule**

To address this issue, one approach is to reduce the learning rate to a very small value, which stabilizes updates near the optimal point and improves convergence. However, this can significantly slow down training, requiring many steps and epochs. A more effective solution is to implement a **learning schedule**, where the learning rate starts high to make rapid progress and gradually decreases over iterations. This balances faster convergence with achieving a solution close to the optimal point.

In the class below, mini-batch Gradient Descent with L1 and L2 regularization and a simple learning schedule function has been implemented.

In [36]:
class MiniBatchGradientDescent:
    """
    Mini-batch Gradient Descent with Elastic Net Regularization for linear regression.
    
    Parameters:
    -----------
    eta : float, default=0.01
        Learning rate. Must be > 0. Large values risk divergence, small values slow convergence.
    
    n_epochs : int, default=1000
        Number of epochs. Must be >= 1. Higher values improve convergence but increase training time.
    
    batch_size : int, default=64
        Mini-batch size. Must be >= 1 and <= dataset size. Smaller batches add noise, larger ones increase computation.
    
    learning_schedule : bool, default=True
        If True, decreases learning rate over iterations.
    
    learning_schedule_t0 : float, default=5
        Controls initial learning rate decay. Must be > 0.
    
    learning_schedule_t1 : float, default=50
        Controls decay rate. Must be > 0.
    
    alpha : float, default=0
        Elastic Net mixing: 0 (L2/Ridge), 1 (L1/Lasso), (0,1) mix. Must be in [0,1].
    
    lambda_ : float, default=0.01
        Regularization strength. Must be >= 0. Higher values increase regularization.
    
    random_state : int, default=42
        Seed for reproducibility.
    """
    def __init__(self, eta=0.01, n_epochs=1000, batch_size=64, learning_schedule=True, 
                 learning_schedule_t0=5, learning_schedule_t1=50, alpha=0, lambda_=0.01, random_state=42):
        self.eta = eta
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.learning_schedule = learning_schedule
        self.learning_schedule_t0 = learning_schedule_t0
        self.learning_schedule_t1 = learning_schedule_t1
        self.alpha = alpha
        self.lambda_ = lambda_
        self.random_state = random_state

    def fit(self, X, y):
        X = add_bias(X)
        y = y.reshape(-1, 1)
        
        np.random.seed(self.random_state)
        self.theta = np.random.rand(X.shape[1], 1)

        def learning_schedule(t):
            return self.learning_schedule_t0 / (t + self.learning_schedule_t1)

        update_count = 0
        
        for epoch in range(self.n_epochs):
            rnd_idx = np.random.permutation(len(X))
            
            for start in range(0, len(X), self.batch_size):
                X_batch = X[rnd_idx[start:start + self.batch_size]]
                y_batch = y[rnd_idx[start:start + self.batch_size]]

                l1_grad = self.lambda_ * self.alpha * np.sign(self.theta)
                l2_grad = self.lambda_ * (1 - self.alpha) * 2 * self.theta
                
                gradients = (2 / X_batch.shape[0]) * X_batch.T @ (X_batch @ self.theta - y_batch) + l1_grad + l2_grad 
                
                if self.learning_schedule:
                    self.eta = learning_schedule(update_count)
                    
                self.theta -= self.eta * gradients
                update_count += 1
                
        return self

    def predict(self, X):
        X = add_bias(X)
        return (X @ self.theta).reshape(-1, 1)

In [37]:
# Testing & Evaluating the model class

minibatchGD = MiniBatchGradientDescent(eta=0.01,n_epochs=4000,batch_size=64,learning_schedule=True, learning_schedule_t0=5,
                                   learning_schedule_t1=50,alpha=0,lambda_=0.01, random_state=42)

evaluate_regression(batchGD, X_train, y_train, X_val, y_val)

-----------------------------------
Val Target True Mean: 57.8266 | Std Dev: 31.1542
RMSE: 1.6950
MAE: 1.3607
R² Score: 0.9970
-----------------------------------


### 2.5. Stochastic Gradient Descent (with Regularization & Learning Schedule)

Stochastic Gradient Descent (SGD) is a variant of Gradient Descent , same as MiniBatchGD, particularly useful when the dataset is too large to fit into memory. By processing only one sample per iteration, SGD is faster than Batch Gradient Descent. However, similar to Mini-batch GD but more pronounced, its path to the minimum is not smooth, exhibiting significant fluctuations and potentially overshooting the minimum due to its high variance. To mitigate this, a learning schedule is implemented to dynamically adjust the learning rate, starting high for rapid progress and decreasing over iterations for better convergence.

In [39]:
class StochasticGradientDescent:
    """
    Stochastic Gradient Descent with Elastic Net Regularization for linear regression.
    
    Parameters:
    -----------
    eta : float, default=0.01
        Learning rate. Must be > 0. Large values risk divergence, small values slow convergence.
    
    n_epochs : int, default=500
        Number of epochs. Must be >= 1. Higher values improve convergence but increase training time.
    
    learning_schedule : bool, default=True
        If True, decreases learning rate over iterations.
    
    learning_schedule_t0 : float, default=5
        Controls initial learning rate decay. Must be > 0.
    
    learning_schedule_t1 : float, default=50
        Controls decay rate. Must be > 0.
    
    alpha : float, default=0
        Elastic Net mixing: 0 (L2/Ridge), 1 (L1/Lasso), (0,1) mix. Must be in [0,1].

    lambda_ : float, default=0.01
        Regularization strength. Must be >= 0. Higher values increase regularization.
    
    random_state : int, default=42
        Seed for reproducibility.
    """
    def __init__(self,eta=0.01,n_epochs=500,learning_schedule=True, learning_schedule_t0=5,learning_schedule_t1=50,alpha=0,
                 lambda_=0.01, random_state=42):
        
        self.eta = eta
        self.n_epochs = n_epochs
        self.learning_schedule = learning_schedule
        self.learning_schedule_t0 = learning_schedule_t0
        self.learning_schedule_t1 = learning_schedule_t1
        self.alpha = alpha
        self.lambda_ = lambda_
        self.random_state = random_state

    def fit(self,X,y):
        X = add_bias(X)
        y = y.reshape(-1,1) # reshape to 1D array

        np.random.seed(self.random_state)
        self.theta = np.random.rand(X.shape[1],1) # an (n,) 1D param vector
        m = X.shape[0]
        
        def learning_schedule(t):
            return self.learning_schedule_t0 / (t + self.learning_schedule_t1)
            
        for epoch in range(self.n_epochs):
            rnd_idx = np.random.permutation(len(X)) # Shuffling indexes
            
            for iteration in range(m):
                xi = X[rnd_idx[iteration]].reshape(-1,1)
                yi = y[rnd_idx[iteration]]

                if self.learning_schedule:
                    self.eta = learning_schedule(epoch * m + iteration)

                l1_grad = self.lambda_ * self.alpha * np.sign(self.theta)
                l2_grad = self.lambda_ * (1-self.alpha) * 2 * self.theta
                
                gradients = 2 * xi * (xi.T @ self.theta - yi) + l1_grad + l2_grad
                self.theta = self.theta - self.eta * gradients
                
        return self

    def predict(self,X):
        X = add_bias(X)
        self.theta.reshape(-1,1)
        yhat = X @ self.theta
        return yhat.reshape(-1,1)

In [42]:
# Testing & Evaluating the model class

stochasticGD = StochasticGradientDescent(eta=0.01,n_epochs=4000,learning_schedule=True, learning_schedule_t0=5,learning_schedule_t1=50,
                                         alpha=0,lambda_=0.01, random_state=42)

evaluate_regression(stochasticGD, X_train, y_train, X_val, y_val)

-----------------------------------
Val Target True Mean: 57.8266 | Std Dev: 31.1542
RMSE: 20.7468
MAE: 16.9764
R² Score: 0.5565
-----------------------------------


## 3. Linear Classification

### 3.1. Logistic Regression (with Regularization & Learning Schedule)

Logistic regression is a fundamental machine learning model for binary classification, predicting the probability of an input belonging to one of two classes (e.g., 0 or 1). The `MiniBatchLogisticRegression` class implements this using mini-batch gradient descent, sigmoid activation, and Elastic Net regularization. 

**Core Idea: Modeling Binary Probabilities**

Logistic regression predicts the probability of an input $\mathbf{x}$ belonging to the positive class (class 1). It computes a linear combination, $\mathbf{\theta}^T\mathbf{x}$, where $\mathbf{x}$ includes a bias term and $\mathbf{\theta}$ is the parameter vector. The **sigmoid function** converts this into a probability:

$$
P(y=1|\mathbf{x}) = \sigma(\mathbf{\theta}^T\mathbf{x}) = \frac{1}{1 + e^{-\mathbf{\theta}^T\mathbf{x}}}
$$

The sigmoid’s S-shape ensures probabilities are between 0 and 1 and provides a smooth gradient for optimization. The `__sigmoid` method implements this, used in `fit` for training and `predict_proba` for probability outputs.

**Loss Function: Log Loss**

To optimize $\mathbf{\theta}$, the model minimizes the **log loss** (negative log-likelihood), which measures how well predicted probabilities match true labels. For $m$ samples, the loss is:

$$
J(\mathbf{\theta}) = -\frac{1}{m} \sum_{i=1}^m \left[ y^{(i)} \log(\sigma(\mathbf{\theta}^T \mathbf{x}^{(i)})) + (1 - y^{(i)}) \log(1 - \sigma(\mathbf{\theta}^T \mathbf{x}^{(i)})) \right]
$$

This penalizes confident incorrect predictions. The `fit` method computes the gradient of this loss to update $\mathbf{\theta}$.

**Gradient Computation and Mini-Batch Optimization**

The gradient of the log loss drives parameter updates:

$$
\nabla_{\theta} J(\theta) = \frac{1}{m} X^T \left( \sigma(X\theta) - y \right) + \lambda \left( \alpha \cdot \text{sign}(\theta) + 2(1 - \alpha) \cdot \theta \right)
$$

This gradient depends on the prediction error $\sigma(X\theta) - y$ and includes regularization terms. The class uses **mini-batch gradient descent**, processing subsets (`batch_size=64`), shuffling data each epoch (`rnd_idx`). The update rule is:

$$
\theta = \theta - \eta \cdot \nabla_{\theta} J(\theta)
$$

A **learning schedule** reduces the learning rate $\eta$:

$$
\eta(t) = \frac{t_0}{t + t_1}
$$

Implemented in `learning_schedule`, this stabilizes convergence using `learning_schedule_t0` and `learning_schedule_t1`.

**Regularization: Controlling Model Complexity**

**Elastic Net regularization** prevents overfitting by adding a penalty to the loss:

$$
J(\theta) = -\frac{1}{m} \sum_{i=1}^m \left[ y^{(i)} \log(\sigma(\mathbf{\theta}^T \mathbf{x}^{(i)})) + (1 - y^{(i)}) \log(1 - \sigma(\mathbf{\theta}^T \mathbf{x}^{(i)})) \right] + \lambda \left( \alpha |\theta| + (1 - \alpha) \theta^2 \right)
$$

- **L1 (Lasso)**: Adds $|\theta|$, promoting sparsity.
- **L2 (Ridge)**: Adds $\theta^2$, encouraging small weights.

Parameters $\lambda$ (`lambda_`) and $\alpha$ (`alpha`) control penalty strength and L1/L2 balance. The `fit` method includes `l1_grad` and `l2_grad` for generalization.

**Decision Boundary and Predictions**

Predictions use a threshold (`decision_threshould=0.5`):

$$
\hat{y} = \begin{cases} 
1 & \text{if } \sigma(\mathbf{\theta}^T\mathbf{x}) \geq \text{threshold} \\
0 & \text{otherwise}
\end{cases}
$$

The `predict` method computes $\sigma(X\theta)$ (`estimated_proba`) and applies the threshold, while `predict_proba` returns probabilities. The decision boundary is at $\mathbf{\theta}^T\mathbf{x} = 0$ for a 0.5 threshold.

This theory underpins the `MiniBatchLogisticRegression` class, enabling robust binary classification.

In [48]:
class LogisticRegression:
    """
    Logistic Regression with Mini-batch Gradient Descent and Elastic Net Regularization.
    
    Parameters:
    -----------
    eta : float, default=0.01
        Learning rate. Must be > 0. Large values risk divergence, small values slow convergence.

    n_epochs : int, default=1000
        Number of epochs. Must be >= 1. Higher values improve convergence but increase training time.
    
    batch_size : int, default=64
        Mini-batch size. Must be >= 1 and <= dataset size. Smaller batches add noise, larger ones increase computation.
    
    decision_threshould : float, default=0.5
        Threshold for classifying probabilities into class labels (0 or 1). Must be in [0, 1].
    
    learning_schedule : bool, default=True
        If True, decreases learning rate over iterations.
    
    learning_schedule_t0 : float, default=5
        Controls initial learning rate decay. Must be > 0.
    
    learning_schedule_t1 : float, default=50
        Controls decay rate. Must be > 0.
    
    alpha : float, default=0
        Elastic Net mixing: 0 (L2/Ridge), 1 (L1/Lasso), (0,1) mix. Must be in [0,1].
    
    lambda_ : float, default=0.01
        Regularization strength. Must be >= 0. Higher values increase regularization.
    
    random_state : int, default=42
        Seed for reproducibility.
    """
    def __init__(self,eta=0.01,n_epochs=1000,batch_size=64,decision_threshould=0.5,learning_schedule=True, 
                 learning_schedule_t0=5,learning_schedule_t1=50,alpha=0,lambda_=0.01, random_state=42):
        self.eta = eta
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.decision_threshould = decision_threshould
        self.learning_schedule = learning_schedule
        self.learning_schedule_t0 = learning_schedule_t0
        self.learning_schedule_t1 = learning_schedule_t1
        self.alpha = alpha
        self.lambda_ = lambda_
        self.random_state = random_state

    def __sigmoid(self,z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self,X,y):
        X = add_bias(X)
        y = y.reshape(-1,1)
        
        np.random.seed(self.random_state)
        self.theta = np.random.rand(X.shape[1],1)

        def learning_schedule(t): # decreasing learning rate over iterations
            return self.learning_schedule_t0 / (t + self.learning_schedule_t1)


        update_count = 0
        
        for epoch in range(self.n_epochs):
            rnd_idx = np.random.permutation(len(X)) # Shuffling indexes
            
            for mini_batch_start in range(0, len(X) , self.batch_size):
                X_batch = X[rnd_idx[mini_batch_start:mini_batch_start+self.batch_size]]
                y_batch = y[rnd_idx[mini_batch_start:mini_batch_start+self.batch_size]]

                l1_grad = self.lambda_ * self.alpha * np.sign(self.theta)
                l2_grad = self.lambda_ * (1-self.alpha) * 2 * self.theta
                
                gradients = (1 / X_batch.shape[0]) * X_batch.T @ (self.__sigmoid(X_batch @ self.theta) - y_batch) + l1_grad + l2_grad 
                
                if self.learning_schedule:
                    self.eta = learning_schedule(update_count)
                    
                self.theta = self.theta - self.eta * gradients
                update_count += 1
                
        return self

    def predict(self,X):
        X = add_bias(X)
        self.estimated_proba = self.__sigmoid( X @ self.theta )
        yhat = (self.estimated_proba >= self.decision_threshould).astype(int)
        return yhat.reshape(-1)

    def predict_proba(self):
        return self.estimated_proba

In [49]:
# Data prepartion for binary classificaion

# Generating data
X , y = generate_poly_classification_data(n_samples=3000, n_features=10,n_classes=2, noise_std=1.0, random_state=42)

# train/val split
X_train , y_train  , X_val , y_val = train_test_split( X , y , random_state=42, train_size=0.8)

# scaling will help faster convergence
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [51]:
# Testing & Evaluating the model class

minbatchLogReg = MiniBatchLogisticRegression(eta=0.01,n_epochs=4000,batch_size=64,decision_threshould=0.5,learning_schedule=True, 
                                 learning_schedule_t0=5,learning_schedule_t1=50,alpha=0,lambda_=0.01, random_state=42)

evaluate_classification(minbatchLogReg, X_train, y_train, X_val, y_val)

-----------------------------------
Accuracy:  0.9583
Precision: 0.9573
Recall:    0.9588
F1 Score:  0.9580
-----------------------------------


### 3.2. Softmax Regression (with Regularization & Learning Schedule)

Softmax regression, also known as multinomial logistic растворregression, extends logistic regression to handle multi-class classification, predicting the probability of an input belonging to one of $k$ classes. The `MiniBatchSoftmaxRegression` class implements this using mini-batch gradient descent, softmax activation, and Elastic Net regularization. Below, we explain the key concepts and formulations, focusing on a machine learning perspective with minimal math, tailored to clarify the provided implementation.

**Core Idea: Modeling Multi-Class Probabilities**

Softmax regression predicts the probability of an input $\mathbf{x}$ belonging to each of $k$ classes. It computes a linear combination for each class, $Z = X\Theta$, where $X$ is the input data (with a bias term) and $\Theta$ is the parameter matrix (weights for each class). The **softmax function** converts these scores into probabilities:

$$
P(y=j|\mathbf{x}) = \text{softmax}(X\Theta)_j = \frac{\exp((X\Theta)_j)}{\sum_{l=1}^k \exp((X\Theta)_l)}
$$

This ensures probabilities sum to 1. The `__softmax` method subtracts the maximum score per sample for numerical stability. The `predict_proba` method outputs these probabilities, while `predict` selects the class with the highest probability using `np.argmax`.

**Loss Function: Cross-Entropy Loss**

To optimize $\Theta$, the model minimizes the **cross-entropy loss**, which measures how well predicted probabilities match true labels. For $m$ samples and $k$ classes, with one-hot encoded labels $Y$, the loss is:

$$
J(\Theta) = -\frac{1}{m} \sum_{i=1}^m \sum_{j=1}^k y_{ij} \log(P(y=j|\mathbf{x}^{(i)}))
$$

Here, $y_{ij} = 1$ if sample $i$ belongs to class $j$, else 0. The `fit` method uses one-hot encoded labels (`y_onehot`) to compute the loss’s gradient.

**Gradient Computation and Mini-Batch Optimization**

The gradient of the cross-entropy loss updates $\Theta$:

$$
\nabla_{\Theta} J(\Theta) = \frac{1}{m} X^T (P - Y) + \lambda \left( \alpha \cdot \text{sign}(\Theta) + 2(1 - \alpha) \cdot \Theta \right)
$$

Here, $P = \text{softmax}(X\Theta)$ is the predicted probability matrix, and $Y$ is the one-hot label matrix. Regularization terms prevent overfitting. The `fit` method computes this gradient for mini-batches (`batch_size=64`), shuffling data each epoch (`rnd_idx`). The update rule is:

$$
\Theta = \Theta - \eta \cdot \nabla_{\Theta} J(\Theta)
$$

A **learning schedule** reduces the learning rate $\eta$:

$$
\eta(t) = \frac{t_0}{t + t_1}
$$

Implemented in `learning_schedule`, this ensures stable convergence using `learning_schedule_t0` and `learning_schedule_t1`.

**Regularization: Controlling Model Complexity**

**Elastic Net regularization** prevents overfitting by adding a penalty to the loss:

$$
J(\Theta) = -\frac{1}{m} \sum_{i=1}^m \sum_{j=1}^k y_{ij} \log(P(y=j|\mathbf{x}^{(i)})) + \lambda \left( \alpha |\Theta| + (1-\alpha) \Theta^2 \right)
$$

- **L1 (Lasso)**: Adds $|\Theta|$, promoting sparsity.
- **L2 (Ridge)**: Adds $\Theta^2$, encouraging small weights.

Parameters $\lambda$ (`lambda_`) and $\alpha$ (`alpha`) control penalty strength and L1/L2 balance. The `fit` method includes `l1_grad` and `l2_grad` for generalization.

**Decision Boundary and Predictions**

The `predict` method computes $X\Theta$ (`logits`), applies softmax, and selects the class with the highest probability. No explicit threshold is needed, unlike logistic regression. The `predict_proba` method returns the probability distribution over classes.

This theory underpins the `MiniBatchSoftmaxRegression` class, enabling robust multi-class classification.

In [56]:
class SoftmaxRegression:
    """
    Softmax Regression with Mini-batch Gradient Descent and Elastic Net Regularization.
    
    Parameters:
    -----------
    eta : float, default=0.01
        Learning rate. Must be > 0. Large values risk divergence, small values slow convergence.
    
    n_epochs : int, default=1000
        Number of epochs. Must be >= 1. Higher values improve convergence but increase training time.
    
    batch_size : int, default=64
        Mini-batch size. Must be >= 1 and <= dataset size. Smaller batches add noise, larger ones increase computation.
    
    learning_schedule : bool, default=True
        If True, decreases learning rate over iterations.
    
    learning_schedule_t0 : float, default=5
        Controls initial learning rate decay. Must be > 0.
    
    learning_schedule_t1 : float, default=50
        Controls decay rate. Must be > 0.
    
    alpha : float, default=0
        Elastic Net mixing: 0 (L2/Ridge), 1 (L1/Lasso), (0,1) mix. Must be in [0,1].
    
    lambda_ : float, default=0.01
        Regularization strength. Must be >= 0. Higher values increase regularization.
    
    random_state : int, default=42
        Seed for reproducibility.
    """
    def __init__(self,eta=0.01,n_epochs=1000,batch_size=64,learning_schedule=True, learning_schedule_t0=5,
                 learning_schedule_t1=50,alpha=0,lambda_=0.01, random_state=42):
        self.eta = eta
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.learning_schedule = learning_schedule
        self.learning_schedule_t0 = learning_schedule_t0
        self.learning_schedule_t1 = learning_schedule_t1
        self.alpha = alpha
        self.lambda_ = lambda_
        self.random_state = random_state

    def __softmax(self,Z):
        Z_stable = Z - np.max(Z, axis=1, keepdims=True) # subtracting max of each row for numerical stability
        exp_Z = np.exp(Z_stable)
        return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

    
    def fit(self,X,y):
        X = add_bias(X)
        y_onehot = OneHotEncoding(y)
        
        np.random.seed(self.random_state)
        self.Theta = np.random.rand(X.shape[1],y_onehot.shape[1])
        
        def learning_schedule(t): # decreasing learning rate over iterations
            return self.learning_schedule_t0 / (t + self.learning_schedule_t1)

        update_count = 0
        
        for epoch in range(self.n_epochs):
            rnd_idx = np.random.permutation(len(X)) # Shuffling indexes
            
            for mini_batch_start in range(0, len(X) , self.batch_size):
                X_batch = X[rnd_idx[mini_batch_start:mini_batch_start+self.batch_size]]
                y_batch = y_onehot[rnd_idx[mini_batch_start:mini_batch_start+self.batch_size]]

                l1_grad = self.lambda_ * self.alpha * np.sign(self.Theta)
                l2_grad = self.lambda_ * (1-self.alpha) * 2 * self.Theta

                gradients = (1 / X_batch.shape[0]) * X_batch.T @ (self.__softmax(X_batch @ self.Theta) - y_batch) + l1_grad + l2_grad 
                
                
                if self.learning_schedule:
                    self.eta = learning_schedule(update_count)
                    
                self.Theta = self.Theta - self.eta * gradients
                update_count += 1
                
        return self

    def predict(self,X):
        X = add_bias(X)
        self.logits = X @ self.Theta
        yhat = np.argmax(self.logits,  axis=1)
        return yhat.reshape(-1,1)

    def predict_proba(self):
        return self.__softmax(self.logits)

In [57]:
# Data prepartion for multi class classificaion

# Generating data
X , y = generate_poly_classification_data(n_samples=3000, n_features=10 , n_classes=4 , noise_std=1.0 , random_state=42)

# train/val split
X_train , y_train  , X_val , y_val = train_test_split( X , y , random_state=42, train_size=0.8)

# scaling will help faster convergence
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [58]:
# Testing & Evaluating the model class

minbatchSoftMaxRegression = MiniBatchSoftmaxRegression(eta=0.01,n_epochs=4000,batch_size=64,learning_schedule=True, learning_schedule_t0=5,
                                                     learning_schedule_t1=50,alpha=0,lambda_=0.01, random_state=42)

evaluate_classification(minbatchSoftMaxRegression, X_train, y_train, X_val, y_val, average="macro")

-----------------------------------
Accuracy:  0.9583
Precision: 0.9723
Recall:    0.6993
F1 Score:  0.8135
-----------------------------------
